<a href="https://colab.research.google.com/github/Haeun-Oh/Deep-Learning/blob/main/9_Check_Point.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.4.1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.4.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.4.1


In [ ]:
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.6MB 56kB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 3.2MB 46.2MB/s 
     |████████████████████████████████| 368kB 44.6MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0


In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'1.13.1'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Uniform distribution 이용해서 변수 초기화
# Adam Optimizer 사용, Dropout 적용

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32) # dropout을 얼마나 시킬 것인지에 대한 placeholder 설정

W1 = tf.Variable(tf.random_uniform([784, 256], -1., 1.))
b1 = tf.Variable(tf.random_uniform([256], -1., 1.))
L1 = tf.nn.sigmoid(tf.matmul(X, W1)+b1)
L1 = tf.nn.dropout(L1, keep_prob) # layer에 dropout 설정

W2 = tf.Variable(tf.random_uniform([256, 256], -1., 1.))
b2 = tf.Variable(tf.random_uniform([256], -1., 1.))
L2 = tf.sigmoid(tf.matmul(L1, W2)+b2)
L2 = tf.nn.dropout(L2,keep_prob)

W3 = tf.Variable(tf.random_uniform([256, 10], -1., 1.))
b3 = tf.Variable(tf.random_uniform([10], -1., 1.))
logits = tf.matmul(L2, W3)+b3
hypothesis = tf.nn.softmax(logits)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
opt = tf.train.AdamOptimizer(
    learning_rate=0.001,
    beta1=0.9,
    beta2=0.999,
    epsilon=1e-08,
    use_locking=False,
    name='Adam').minimize(cost)

batch_size = 100

ckpt_path = '/content/gdrive/MyDrive/Colab Notebooks/Deep_Learning/9. Check Point/saved/adam.ckpt'
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    for epoch in range(15):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            c, _ = sess.run([cost, opt], feed_dict={X:batch_xs, Y:batch_ys, keep_prob: 0.75}) #75% node를 활성화 시키겠다.
            avg_cost += c/total_batch
        print('Epoch:', '%d' % (epoch+1), 'cost = ', '{:.9f}'.format(avg_cost))      
    is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print("Accuracy", sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob: 1}))   

    saver.save(sess, ckpt_path)
    saver.restore(sess, ckpt_path)           
    print("Accuracy", sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob: 1}))              

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Epoch: 1 cost =  2.497446042
Epoch: 2 cost =  0.890527407
Epoch: 3 cost =  0.624555755
Epoch: 4 cost =  0.493519641
Epoch: 5 cost =  0.406260532
Epoch: 6 cost =  0.337335690
Epoch: 7 cost =  0.294697071
Epoch: 8 cost =  0.262131140
Epoch: 9 cost =  0.231153402
Epoch: 10 cost =  0.208078405
Epoch: 11 cost =  0.194845731
Epoch: 12 cost =  0.173534792
Epoch: 13 cost =  0.163723611
Epoch: 14 cost =  0.150468963
Epoch: 15 cost =  0.135852340
Accuracy 0.9682
INFO:tensorflow:Restoring parameters from /content/gdrive/MyDrive/Colab Notebooks/Deep_Learning/9. Check Point/saved/adam.ckpt
Accuracy 0.9682


In [ ]:
# Xavier 이용해서 변수 초기화
# Adam Optimizer 사용, Dropout 적용

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.get_variable("W1", shape=[784, 256],initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.get_variable("b1", shape=[256],initializer=tf.contrib.layers.xavier_initializer())
L1 = tf.nn.sigmoid(tf.matmul(X, W1)+b1)
L1 = tf.nn.dropout(L1, keep_prob) 

W2 = tf.get_variable("W2", shape=[256, 256],initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable("b2", shape=[256],initializer=tf.contrib.layers.xavier_initializer())
L2 = tf.sigmoid(tf.matmul(L1, W2)+b2)
L2 = tf.nn.dropout(L2,keep_prob)

W3 = tf.get_variable("W3", shape=[256, 10],initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.get_variable("b3", shape=[10],initializer=tf.contrib.layers.xavier_initializer())
logits = tf.matmul(L2, W3)+b3
hypothesis = tf.nn.softmax(logits)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
opt = tf.train.AdamOptimizer(
    learning_rate=0.001,
    beta1=0.9,
    beta2=0.999,
    epsilon=1e-08,
    use_locking=False,
    name='Adam').minimize(cost)

batch_size = 100

ckpt_path = '/content/gdrive/MyDrive/Colab Notebooks/Deep_Learning/9. Check Point/saved/adam.ckpt'
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    for epoch in range(15):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            c, _ = sess.run([cost, opt], feed_dict={X:batch_xs, Y:batch_ys, keep_prob: 0.75}) #75% node를 활성화 시키겠다.
            avg_cost += c/total_batch
        print('Epoch:', '%d' % (epoch+1), 'cost = ', '{:.9f}'.format(avg_cost))      
    is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print("Accuracy", sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob: 1}))   

    saver.save(sess, ckpt_path)
    saver.restore(sess, ckpt_path)           
    print("Accuracy", sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob: 1}))

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch: 1 cost =  0.636390437
Epoch: 2 cost =  0.272589289
Epoch: 3 cost =  0.212541032
Epoch: 4 cost =  0.174185623
Epoch: 5 cost =  0.144472443
Epoch: 6 cost =  0.124314506
Epoch: 7 cost =  0.107521676
Epoch: 8 cost =  0.095830982
Epoch: 9 cost =  0.086667126
Epoch: 10 cost =  0.078764851
Epoch: 11 cost =  0.071442220
Epoch: 12 cost =  0.063400061
Epoch: 13 cost =  0.058437356
Epoch: 14 cost =  0.053292058
Epoch: 15 cost =  0.049998201
Accuracy 0.9806
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/gdrive/MyDrive/C